In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cropdisease2/crop2/manifest-8517f262e2ca61a7f7ac8fc1d32eb0be20241003-7123-1lvxb66.json
/kaggle/input/cropdisease2/crop2/SampleSubmission.csv
/kaggle/input/cropdisease2/crop2/Rail_Challenge_Starter.ipynb
/kaggle/input/cropdisease2/crop2/Train.csv
/kaggle/input/cropdisease2/crop2/Test.csv
/kaggle/input/cropdisease2/crop2/images/id_p8dxgn.jpg
/kaggle/input/cropdisease2/crop2/images/id_g2syp1.jpg
/kaggle/input/cropdisease2/crop2/images/id_zx8php.jpg
/kaggle/input/cropdisease2/crop2/images/id_6ne6qc.jpg
/kaggle/input/cropdisease2/crop2/images/id_masrex.jpg
/kaggle/input/cropdisease2/crop2/images/id_v9lx4u.jpg
/kaggle/input/cropdisease2/crop2/images/id_qn4cyl.jpg
/kaggle/input/cropdisease2/crop2/images/id_q97obi.jpg
/kaggle/input/cropdisease2/crop2/images/id_vxaj7n.jpg
/kaggle/input/cropdisease2/crop2/images/id_xh32sn.jpg
/kaggle/input/cropdisease2/crop2/images/id_926bx7.jpg
/kaggle/input/cropdisease2/crop2/images/id_2wvsgz.jpg
/kaggle/input/cropdisease2/crop2/images/id_z5is61.

In [3]:
!pip -q install ultralytics

In [4]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import shutil
import yaml
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from ultralytics import YOLO
import albumentations as A
from tqdm.auto import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
import warnings
import gc
warnings.filterwarnings('ignore')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [5]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128,expandable_segments:True'

In [6]:
class Config:
    
    BASE_PATH = Path('/kaggle/input/cropdisease2/crop2')
    TRAIN_CSV = BASE_PATH / 'Train.csv'
    TEST_CSV = BASE_PATH / 'Test.csv'
    IMAGES_DIR = BASE_PATH / 'images'
    
    
    DATASET_DIR = Path('/kaggle/working/dataset')
    TRAIN_DIR = DATASET_DIR / 'train' / 'images'
    TRAIN_LABELS_DIR = DATASET_DIR / 'train' / 'labels'
    VAL_DIR = DATASET_DIR / 'valid' / 'images'
    VAL_LABELS_DIR = DATASET_DIR / 'valid' / 'labels'
    
    
    IMG_SIZE = 416  
    BATCH_SIZE = 16
    NUM_WORKERS = 4
    VAL_RATIO = 0.2
    

    EPOCHS = 50
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    SEED = 42
    GRADIENT_ACCUMULATION = 4
    AMP = True

In [7]:
np.random.seed(Config.SEED)
torch.manual_seed(Config.SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(Config.SEED)
    torch.cuda.manual_seed_all(Config.SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

dirs = [
    Config.DATASET_DIR,
    Config.TRAIN_DIR,
    Config.TRAIN_LABELS_DIR,
    Config.VAL_DIR,
    Config.VAL_LABELS_DIR
]
for dir_path in dirs:
    if dir_path.exists():
        shutil.rmtree(dir_path)
    dir_path.mkdir(parents=True, exist_ok=True)

In [8]:
def convert_to_yolo_format(row, img_width, img_height):
    """Convert bounding box coordinates to YOLO format."""
    x_center = ((row['xmin'] + row['xmax']) / 2) / img_width
    y_center = ((row['ymin'] + row['ymax']) / 2) / img_height
    width = (row['xmax'] - row['xmin']) / img_width
    height = (row['ymax'] - row['ymin']) / img_height
    
    return f"{row['class_id']} {x_center} {y_center} {width} {height}"

In [9]:
print("Reading data...")
train_df = pd.read_csv(Config.TRAIN_CSV)
test_df = pd.read_csv(Config.TEST_CSV)

Reading data...


In [10]:
class_mapping = {cls: idx for idx, cls in enumerate(sorted(train_df['class'].unique()))}
train_df['class_id'] = train_df['class'].map(class_mapping)

In [11]:
unique_images = train_df['Image_ID'].unique()
np.random.shuffle(unique_images)
split_idx = int(len(unique_images) * (1 - Config.VAL_RATIO))
train_images = unique_images[:split_idx]
val_images = unique_images[split_idx:]

In [12]:
print("Processing training data...")
batch_size = 400
for i in range(0, len(train_images), batch_size):
    batch_images = train_images[i:i + batch_size]
    
    for img_id in tqdm(batch_images, leave=False):
        src_img_path = Config.IMAGES_DIR / img_id
        dst_img_path = Config.TRAIN_DIR / img_id
        shutil.copy(src_img_path, dst_img_path)
        
        img_data = train_df[train_df['Image_ID'] == img_id]
        img = cv2.imread(str(src_img_path))
        height, width = img.shape[:2]
        del img  
        
        label_path = Config.TRAIN_LABELS_DIR / f"{Path(img_id).stem}.txt"
        with open(label_path, 'w') as f:
            for _, row in img_data.iterrows():
                yolo_format = convert_to_yolo_format(row, width, height)
                f.write(f"{yolo_format}\n")
    
    gc.collect()
    torch.cuda.empty_cache()

Processing training data...


  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

...

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/321 [00:00<?, ?it/s]

In [13]:
print("Processing validation data...")
for i in range(0, len(val_images), batch_size):
    batch_images = val_images[i:i + batch_size]
    
    for img_id in tqdm(batch_images, leave=False):
        src_img_path = Config.IMAGES_DIR / img_id
        dst_img_path = Config.VAL_DIR / img_id
        shutil.copy(src_img_path, dst_img_path)
        
        img_data = train_df[train_df['Image_ID'] == img_id]
        img = cv2.imread(str(src_img_path))
        height, width = img.shape[:2]
        del img  
        
        label_path = Config.VAL_LABELS_DIR / f"{Path(img_id).stem}.txt"
        with open(label_path, 'w') as f:
            for _, row in img_data.iterrows():
                yolo_format = convert_to_yolo_format(row, width, height)
                f.write(f"{yolo_format}\n")
    
    gc.collect()
    torch.cuda.empty_cache()

Processing validation data...


  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/181 [00:00<?, ?it/s]

In [14]:
data_yaml = {
    'path': str(Config.DATASET_DIR),
    'train': str(Config.TRAIN_DIR),
    'val': str(Config.VAL_DIR),
    'nc': len(class_mapping),
    'names': list(class_mapping.keys())
}

yaml_path = Config.DATASET_DIR / 'data.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

In [15]:
print("Training model...")
model = YOLO('yolov8n.pt')

Training model...


100%|██████████| 6.25M/6.25M [00:00<00:00, 24.3MB/s]


In [16]:
train_args = {
    'data': str(yaml_path),
    'epochs': Config.EPOCHS,
    'imgsz': Config.IMG_SIZE,
    'batch': Config.BATCH_SIZE,
    'device': Config.DEVICE,
    'workers': Config.NUM_WORKERS,
    'patience': 10,
    'cos_lr': True,
    'lr0': 0.01,
    'lrf': 0.001,
    'warmup_epochs': 3,
    'warmup_momentum': 0.5,
    'box': 7.5,
    'cls': 0.5,
    'dfl': 1.5,
    
    
    'mixup': 0.0,
    'copy_paste': 0.0,
    'cache': False,
    'mosaic': 0.0,
    'rect': True,
    'amp': True,
    
    
    'augment': True,
    'degrees': 10.0,
    'translate': 0.1,
    'scale': 0.5,
    'shear': 2.0,
    'perspective': 0.0,
    'flipud': 0.5,
    'fliplr': 0.5,
    
    'optimizer': 'AdamW',
    'close_mosaic': 0,
    'overlap_mask': False
}


model.train(**train_args)

Ultralytics 8.3.27 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/dataset/data.yaml, epochs=50, time=None, patience=10, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=cuda, workers=4, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=True, cos_lr=True, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=False, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, 

100%|██████████| 755k/755k [00:00<00:00, 4.64MB/s]
2024-11-05 03:35:49,375	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-05 03:35:49,895	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=23

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 22.3MB/s]


AMP: checks passed ✅


...

      50/50      1.02G      2.487      2.634      1.562          3        320: 100%|██████████| 246/246 [01:55<00:00,  2.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:24<00:00,  1.27it/s]


                   all        981       8225      0.505      0.108     0.0654     0.0247

50 epochs completed in 1.960 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.27 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 168 layers, 3,010,133 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:24<00:00,  1.26it/s]


                   all        981       8225      0.501     0.0988     0.0671     0.0255
Corn_Cercospora_Leaf_Spot        157       1221          1          0     0.0445     0.0201
      Corn_Common_Rust         50        276          1          0     0.0881      0.028
          Corn_Healthy         51        321          1          0      0.128      0.047
Corn_Northern_Leaf_Blight         13         34     0.0271     0.0588    0.00612    0.00278
           Corn_Streak         73        636      0.093      0.574       0.18     0.0574
 Pepper_Bacterial_Spot        106        415      0.146     0.0747     0.0516      0.013
     Pepper_Cercospora         42        107          0          0     0.0195     0.0039
       Pepper_Fusarium         40         89      0.133      0.191     0.0635     0.0208
        Pepper_Healthy         62        155      0.167      0.458      0.189     0.0831
    Pepper_Late_Blight         10        137          1          0          0          0
    Pepper_Leaf

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d21e819b520>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042, 

In [28]:
import os


output_dir = '/kaggle/output'


os.makedirs(output_dir, exist_ok=True)


In [29]:
model.save(f'{output_dir}/output.pt')

In [30]:
model=YOLO('runs/detect/train/weights/best.pt')
results=model.val()

Ultralytics 8.3.27 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 168 layers, 3,010,133 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/dataset/valid/labels.cache... 981 images, 0 backgrounds, 0 corrupt: 100%|██████████| 981/981 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [00:21<00:00,  2.83it/s]


                   all        981       8225      0.505      0.108     0.0657     0.0247
Corn_Cercospora_Leaf_Spot        157       1221          1          0      0.037     0.0161
      Corn_Common_Rust         50        276          1          0     0.0829     0.0239
          Corn_Healthy         51        321          1          0       0.11     0.0479
Corn_Northern_Leaf_Blight         13         34     0.0205     0.0882    0.00458    0.00123
           Corn_Streak         73        636      0.079      0.588      0.179     0.0559
 Pepper_Bacterial_Spot        106        415       0.15     0.0867     0.0546     0.0143
     Pepper_Cercospora         42        107          0          0     0.0233    0.00464
       Pepper_Fusarium         40         89      0.117      0.196     0.0605      0.016
        Pepper_Healthy         62        155      0.155      0.484      0.175      0.076
    Pepper_Late_Blight         10        137          1          0          0          0
    Pepper_Leaf

In [31]:
import shutil


source_dir = 'runs/detect/val'
dest_dir = '/kaggle/working/val_results'


shutil.copytree(source_dir, dest_dir)

'/kaggle/working/val_results'

In [32]:
import shutil

# Define source and destination directories
source_dir = 'runs/'
dest_dir = '/kaggle/working/outputs'

# Copy the directory to the working directory
shutil.copytree(source_dir, dest_dir)

'/kaggle/working/outputs'

In [33]:
print("Making predictions...")
predictions = []
test_batch_size = 16

for i in range(0, len(test_df), test_batch_size):
    batch_df = test_df.iloc[i:i + test_batch_size]
    batch_predictions = []
    
    for _, row in tqdm(batch_df.iterrows(), leave=False):
        img_path = Config.IMAGES_DIR / row['Image_ID']
        with torch.cuda.amp.autocast(enabled=True):
            results = model(str(img_path))
        
        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy()
            classes = result.boxes.cls.cpu().numpy()
            confs = result.boxes.conf.cpu().numpy()
            
            for box, cls, conf in zip(boxes, classes, confs):
                batch_predictions.append({
                    'Image_ID': row['Image_ID'],
                    'class': result.names[int(cls)],
                    'confidence': conf,
                    'ymin': box[1],
                    'xmin': box[0],
                    'ymax': box[3],
                    'xmax': box[2]
                })
    
    predictions.extend(batch_predictions)
    del batch_predictions
    torch.cuda.empty_cache()
    gc.collect()

Making predictions...


0it [00:00, ?it/s]


image 1/1 /kaggle/input/cropdisease2/crop2/images/id_128pxx.jpg: 288x416 (no detections), 46.3ms
Speed: 1.3ms preprocess, 46.3ms inference, 1.3ms postprocess per image at shape (1, 3, 288, 416)

image 1/1 /kaggle/input/cropdisease2/crop2/images/id_12jbci.jpg: 288x416 (no detections), 8.9ms
Speed: 1.0ms preprocess, 8.9ms inference, 0.5ms postprocess per image at shape (1, 3, 288, 416)

image 1/1 /kaggle/input/cropdisease2/crop2/images/id_143s4o.jpg: 192x416 (no detections), 45.5ms
Speed: 1.0ms preprocess, 45.5ms inference, 0.5ms postprocess per image at shape (1, 3, 192, 416)

image 1/1 /kaggle/input/cropdisease2/crop2/images/id_14tfmb.jpg: 320x416 (no detections), 48.8ms
Speed: 1.8ms preprocess, 48.8ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /kaggle/input/cropdisease2/crop2/images/id_14tw4o.jpg: 320x416 1 Tomato_Early_Blight, 9.5ms
Speed: 1.6ms preprocess, 9.5ms inference, 2.9ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /kaggle/inpu

0it [00:00, ?it/s]


image 1/1 /kaggle/input/cropdisease2/crop2/images/id_1b3zwu.jpg: 320x416 (no detections), 10.0ms
Speed: 1.7ms preprocess, 10.0ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /kaggle/input/cropdisease2/crop2/images/id_1ba1dt.jpg: 192x416 (no detections), 9.9ms
Speed: 1.1ms preprocess, 9.9ms inference, 0.5ms postprocess per image at shape (1, 3, 192, 416)

image 1/1 /kaggle/input/cropdisease2/crop2/images/id_1bkn2l.jpg: 320x416 2 Tomato_Healthys, 10.1ms
Speed: 1.6ms preprocess, 10.1ms inference, 1.3ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /kaggle/input/cropdisease2/crop2/images/id_1cl2pj.jpg: 288x416 (no detections), 9.7ms
Speed: 1.1ms preprocess, 9.7ms inference, 0.5ms postprocess per image at shape (1, 3, 288, 416)

image 1/1 /kaggle/input/cropdisease2/crop2/images/id_1do3cp.jpg: 320x416 (no detections), 10.1ms
Speed: 1.8ms preprocess, 10.1ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /kaggle/input/cr

...


image 1/1 /kaggle/input/cropdisease2/crop2/images/id_kr9trr.jpg: 320x416 (no detections), 9.9ms
Speed: 1.7ms preprocess, 9.9ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /kaggle/input/cropdisease2/crop2/images/id_lpypdz.jpg: 320x416 12 Tomato_Healthys, 9.4ms
Speed: 1.6ms preprocess, 9.4ms inference, 1.3ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /kaggle/input/cropdisease2/crop2/images/id_o813rn.jpg: 320x416 (no detections), 9.5ms
Speed: 1.5ms preprocess, 9.5ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /kaggle/input/cropdisease2/crop2/images/id_inkn3f.jpg: 288x416 (no detections), 10.0ms
Speed: 1.1ms preprocess, 10.0ms inference, 0.5ms postprocess per image at shape (1, 3, 288, 416)

image 1/1 /kaggle/input/cropdisease2/crop2/images/id_tw35sy.jpg: 320x416 (no detections), 9.6ms
Speed: 1.5ms preprocess, 9.6ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /kaggle/input/cropd

0it [00:00, ?it/s]


image 1/1 /kaggle/input/cropdisease2/crop2/images/id_4mdsot.jpg: 320x416 1 Pepper_Healthy, 10.0ms
Speed: 1.8ms preprocess, 10.0ms inference, 1.3ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /kaggle/input/cropdisease2/crop2/images/id_frhr24.jpg: 320x416 (no detections), 9.3ms
Speed: 1.6ms preprocess, 9.3ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /kaggle/input/cropdisease2/crop2/images/id_opc4t1.jpg: 192x416 (no detections), 9.9ms
Speed: 1.0ms preprocess, 9.9ms inference, 0.5ms postprocess per image at shape (1, 3, 192, 416)

image 1/1 /kaggle/input/cropdisease2/crop2/images/id_xdd1xo.jpg: 320x416 (no detections), 9.6ms
Speed: 1.6ms preprocess, 9.6ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /kaggle/input/cropdisease2/crop2/images/id_42cen6.jpg: 320x416 (no detections), 10.0ms
Speed: 1.7ms preprocess, 10.0ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /kaggle/input/cropd

0it [00:00, ?it/s]


image 1/1 /kaggle/input/cropdisease2/crop2/images/id_iytwrm.jpg: 320x416 (no detections), 9.9ms
Speed: 1.4ms preprocess, 9.9ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /kaggle/input/cropdisease2/crop2/images/id_5sfgyq.jpg: 320x416 (no detections), 9.2ms
Speed: 1.6ms preprocess, 9.2ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /kaggle/input/cropdisease2/crop2/images/id_5s6bb8.jpg: 320x416 (no detections), 9.2ms
Speed: 1.6ms preprocess, 9.2ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /kaggle/input/cropdisease2/crop2/images/id_sp7ucp.jpg: 320x416 (no detections), 9.1ms
Speed: 1.6ms preprocess, 9.1ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /kaggle/input/cropdisease2/crop2/images/id_fdgccx.jpg: 320x416 (no detections), 9.3ms
Speed: 1.6ms preprocess, 9.3ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 416)


In [34]:
predictions_df = pd.DataFrame(predictions)
predictions_df.to_csv('submission.csv', index=False)
print("Predictions saved to submission.csv")

Predictions saved to submission.csv


In [35]:
predictions_df=pd.read_csv('submission.csv')

In [36]:
predictions_df.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,id_14tw4o.jpg,Tomato_Early_Blight,0.368408,1395.14430,2047.3558,1753.1250,2405.3367
1,id_15ozvw.jpg,Pepper_Bacterial_Spot,0.259766,1469.92800,2030.1924,1587.6202,2147.8848
2,id_167cbn.jpg,Tomato_Healthy,0.322510,276.05768,1095.0000,530.4808,1450.3846
3,id_18wf25.jpg,Corn_Streak,0.322510,2689.61520,1041.9231,3024.0000,1625.8845
4,id_1bkn2l.jpg,Tomato_Healthy,0.373047,231.40384,866.2500,1315.7307,1553.1923


In [37]:
predictions_df.describe()

,confidence,ymin,xmin,ymax,xmax
count,647.000000,647.000000,647.000000,647.000000,647.000000
mean,0.319764,1107.845514,1418.540687,1743.535504,2028.545891
std,0.080141,716.676694,1023.084718,821.142574,1107.073561
min,0.250244,0.000000,0.000000,81.724754,196.442300
25%,0.267334,504.483180,639.209125,1070.026450,1065.576900
50%,0.294189,1062.307700,1234.557600,1760.365400,1913.726000
75%,0.334961,1640.350950,2085.360600,2434.947000,2822.509650
max,0.823242,3737.596200,4700.769500,4480.000000,5613.462000


In [38]:
predictions_df['class'].value_counts()

class
Tomato_Healthy               336
Corn_Streak                  133
Pepper_Healthy                83
Tomato_Early_Blight           35
Tomato_Fusarium               21
Pepper_Bacterial_Spot         16
Pepper_Fusarium               13
Tomato_Septoria                7
Corn_Northern_Leaf_Blight      1
Pepper_Leaf_Blight             1
Tomato_Late_Blight             1
Name: count, dtype: int64

In [39]:
predictions_df.to_csv('/kaggle/working/predictions_df.csv',index=False)